In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121, MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import os


In [2]:
dataset_path = "https://drive.google.com/drive/folders/1qfAKL-V2_tIpFFNo4lyZ-hcAwPg1lt_G?usp=drive_link"
train_path = os.path.join(dataset_path, "Train")
test_path = os.path.join(dataset_path, "Test")


In [3]:
image_size = (224, 224)

base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


# Veri Setinin Eğitim/Validasyon/Test Olarak Ayrılması

In [4]:
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 31411 images belonging to 43 classes.
Found 7848 images belonging to 43 classes.
Found 0 images belonging to 0 classes.


# Modelin Eğitilmesi

In [5]:
predictions = Dense(43, activation='softmax')(x)  # Changed from 2 to 43

model = Model(inputs=base_model.input, outputs=predictions)

In [6]:
from tensorflow.keras.applications import MobileNetV2

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
import tensorflow as tf
print("GPU Kullanılıyor mu?:", tf.config.list_physical_devices('GPU'))


GPU Kullanılıyor mu?: []


In [10]:
train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=64,  # Varsayılan genelde 32, burada 64 yapıldı.
    class_mode='categorical',
    subset='training'
)


Found 31411 images belonging to 43 classes.


In [11]:
model.compile(
    optimizer=Adam(learning_rate=0.005),  # Varsayılan 0.001 yerine daha büyük bir değer
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [12]:
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=5,  # Daha az dönüşüm
    width_shift_range=0.05,  # Daha az kaydırma
    height_shift_range=0.05,
    zoom_range=0.05,
    horizontal_flip=False,  # Yüz maskesi için simetriyi bozmayalım
    validation_split=0.2
)


In [13]:
for layer in base_model.layers[:100]:
    layer.trainable = False  # İlk 100 katmanı dondur


In [16]:
# ... previous code ...

# Define the model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
# Change the number of output neurons to match the number of classes (43)
predictions = Dense(43, activation='softmax')(x)  # Changed from 2 to 43

model = Model(inputs=base_model.input, outputs=predictions)

# ... rest of your code ...

In [19]:
import tensorflow as tf
import os

# Get the TPU address from the environment variable 'COLAB_TPU_ADDR'
try:
    # Check if TPU is available in Colab environment
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None
    print('TPU not found, using CPU/GPU instead')

# Connect to and initialize the TPU system if a TPU was found
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU

# Rest of your code...

TPU not found, using CPU/GPU instead


In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6)

history = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/5
298/491 ━━━━━━━━━━━━━━━━━━━━ 1:10:07 22s/step - accuracy: 0.6100 - loss: 1.5165

# Modelin Performans Analizi

In [ ]:
# Eğitim ve doğrulama kayıpları ve doğrulukları görselleştirme
plt.figure(figsize=(12, 6))

# Accuracy grafiği
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss grafiği
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


# Modelin Test Verisi Üzerinde Değerlendirilmesi

In [ ]:
# En iyi ağırlıkları yükle
model.load_weights("best_model.h5")

# Test verisi ile tahmin yap
y_pred_prob = model.predict(test_generator)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = test_generator.classes

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# Performans metrikleri
print(classification_report(y_true, y_pred))

# ROC ve AUC Hesaplama
fpr, tpr, _ = roc_curve(y_true, y_pred_prob[:, 1])
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='grey', linestyle='--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend()
plt.show()
